In [22]:
import pandas as pd
import numpy as np

## Austin

In [ ]:
# 合併所有結果
austin_df = pd.concat(all_matched, ignore_index=True) if all_matched else pd.DataFrame()

print(f"Total unique animals: {len(unique_ids)}")
print(f"Matched records: {len(austin_df)}")
print(f"Animals with multiple records: {austin_df['Animal ID'].duplicated().sum()}")
print(f"Records with conflicts: {len(all_conflicts)}")

# 顯示衝突摘要
if all_conflicts:
    print("\nConflict Summary (只顯示兩個值都非空且不同的情況):")
    for i, conflict in enumerate(all_conflicts[:5]):  # 顯示前5個衝突
        print(f"Animal {conflict['Animal ID']}:")
        for col in consistency_cols:
            if col in conflict:
                print(f"  {col}: Intake='{conflict[col]['intake']}' vs Outcome='{conflict[col]['outcome']}'")
        print()

# 儲存結果
austin_df.to_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/processed/austin.csv', index=False)
austin_df

In [ ]:
austin_intake_cleaned = (
    austin_intake
    .drop(columns=['MonthYear'])
    .rename(columns={'DateTime':'Intake_DateTime'})
)

austin_outcome_cleaned = (
    austin_outcome
    .drop(columns=['MonthYear'])
    .rename(columns={'DateTime':'Outcome_DateTime'})
)

# 轉換時間格式
austin_intake_cleaned['Intake_DateTime'] = pd.to_datetime(austin_intake_cleaned['Intake_DateTime'])
austin_outcome_cleaned['Outcome_DateTime'] = pd.to_datetime(austin_outcome_cleaned['Outcome_DateTime'])

# 需要檢查一致性的欄位
consistency_cols = ['Animal Type', 'Breed', 'Color']

# 為了提升效能，先按 Animal ID 分組
intake_grouped = austin_intake_cleaned.groupby('Animal ID')
outcome_grouped = austin_outcome_cleaned.groupby('Animal ID')

all_matched = []
all_conflicts = []

# 取得所有唯一的 Animal ID
unique_ids = set(austin_intake_cleaned['Animal ID'].unique()) | set(austin_outcome_cleaned['Animal ID'].unique())

for animal_id in unique_ids:
    # 快速取得該動物的記錄
    try:
        intake_records = intake_grouped.get_group(animal_id).sort_values('Intake_DateTime')
    except KeyError:
        intake_records = pd.DataFrame(columns=austin_intake_cleaned.columns)
    
    try:
        outcome_records = outcome_grouped.get_group(animal_id).sort_values('Outcome_DateTime')
    except KeyError:
        outcome_records = pd.DataFrame(columns=austin_outcome_cleaned.columns)
    
    # 如果沒有 intake 記錄，跳過
    if intake_records.empty:
        continue
    
    matched_records = []
    outcome_used = set()  # 使用 set 來追蹤已使用的 outcome 記錄索引
    
    for _, intake in intake_records.iterrows():
        # 若完全沒有 outcome 記錄，直接保留 intake
        if outcome_records.empty:
            matched_records.append(intake)
            continue
        
        available_outcomes = outcome_records.loc[~outcome_records.index.isin(outcome_used)]
        
        # 若 outcome 都被使用過了，也直接保留 intake
        if available_outcomes.empty:
            matched_records.append(intake)
            continue
        
        # 找到在這次 Intake 之後的第一個未使用的 Outcome
        future_outcomes = available_outcomes[
            available_outcomes['Outcome_DateTime'] >= intake['Intake_DateTime']
        ]
        
        if not future_outcomes.empty:
            # 配對到最早的 Outcome
            matched_outcome = future_outcomes.iloc[0]
            outcome_used.add(matched_outcome.name)
            
            # 合併記錄
            merged_record = intake.copy()
            conflicts = {}
            
            # 檢查一致性並合併欄位
            for col in consistency_cols:
                intake_val = intake.get(col)
                outcome_val = matched_outcome.get(col)
                
                # 只有當兩個值都非空且不同時才算衝突
                if pd.notna(intake_val) and pd.notna(outcome_val) and intake_val != outcome_val:
                    conflicts[col] = {'intake': intake_val, 'outcome': outcome_val}
                    # 優先使用 Intake 的值
                    merged_record[col] = intake_val
                else:
                    # 使用非空值，優先 Intake
                    merged_record[col] = intake_val if pd.notna(intake_val) else outcome_val
            
            # 添加 Outcome 特定的欄位
            for col in matched_outcome.index:
                if col not in consistency_cols and col != 'Animal ID':
                    if col.endswith('_DateTime'):
                        merged_record[col] = matched_outcome[col]
                    elif col not in merged_record:
                        merged_record[col] = matched_outcome[col]
                    elif pd.isna(merged_record[col]) and pd.notna(matched_outcome[col]):
                        merged_record[col] = matched_outcome[col]
                    elif (pd.notna(merged_record[col]) and pd.notna(matched_outcome[col]) and 
                          merged_record[col] != matched_outcome[col]):
                        # 如果有不同值，保留 Outcome 版本
                        merged_record[f"{col}_Outcome"] = matched_outcome[col]
            
            if conflicts:
                conflicts['Animal ID'] = animal_id
                conflicts['Intake_DateTime'] = intake['Intake_DateTime']
                conflicts['Outcome_DateTime'] = matched_outcome['Outcome_DateTime']
                all_conflicts.append(conflicts)
            
            matched_records.append(merged_record)
        else:
            # 沒有對應的 Outcome，只保留 Intake 記錄
            matched_records.append(intake)
    
    if matched_records:
        all_matched.append(pd.DataFrame(matched_records))

# 合併所有結果
austin = pd.concat(all_matched, ignore_index=True) if all_matched else pd.DataFrame()

print(f"Total unique animals: {len(unique_ids)}")
print(f"Matched records: {len(austin)}")
print(f"Animals with multiple records: {austin['Animal ID'].duplicated().sum()}")
print(f"Records with conflicts: {len(all_conflicts)}")

# 顯示衝突摘要
if all_conflicts:
    print("\nConflict Summary (只顯示兩個值都非空且不同的情況):")
    for i, conflict in enumerate(all_conflicts[:5]):  # 顯示前5個衝突
        print(f"Animal {conflict['Animal ID']}:")
        for col in consistency_cols:
            if col in conflict:
                print(f"  {col}: Intake='{conflict[col]['intake']}' vs Outcome='{conflict[col]['outcome']}'")
        print()

# 儲存結果
austin.to_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/processed/austin.csv', index=False)
austin

/var/folders/sh/9h4tb2t16kn57sd6_g6xwsbm0000gn/T/ipykernel_6383/2199020215.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  austin_intake_cleaned['Intake_DateTime'] = pd.to_datetime(austin_intake_cleaned['Intake_DateTime'])
/var/folders/sh/9h4tb2t16kn57sd6_g6xwsbm0000gn/T/ipykernel_6383/2199020215.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  austin_outcome_cleaned['Outcome_DateTime'] = pd.to_datetime(austin_outcome_cleaned['Outcome_DateTime'])
/var/folders/sh/9h4tb2t16kn57sd6_g6xwsbm0000gn/T/ipykernel_6383/2199020215.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a fo

Total unique animals: 111740
Matched records: 124120
Animals with multiple records: 13194
Records with conflicts: 0


,Animal ID,Name,Intake_DateTime,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Outcome_DateTime,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome
0,A813443,Oreo Cookie,2020-02-10 14:36:00,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,12 years,Domestic Shorthair,Black/White,2020-02-14 14:04:00,02/10/2008,Transfer,Partner,Spayed Female,12 years
1,A735087,NaN,2016-09-18 10:52:00,14219 Littig Rd in Manor (TX),Stray,Injured,Dog,Spayed Female,3 years,Rhod Ridgeback/Pointer,Tan/White,2016-09-22 12:37:00,09/18/2013,Adoption,NaN,Spayed Female,3 years
2,A677085,NaN,2014-04-19 12:47:00,11629 Timber Heights in Austin (TX),Stray,Normal,Other,Unknown,1 year,Rabbit Sh Mix,Black,2014-04-20 10:47:00,04/19/2013,Transfer,Partner,Unknown,1 year
3,A737847,*Lena,2016-11-05 14:15:00,E Martin Luther King Jr Blvd & Chicon St in Au...,Stray,Normal,Dog,Intact Female,1 year,Pit Bull Mix,Blue/White,2016-11-11 14:35:00,11/05/2015,Transfer,Partner,Intact Female,1 year
4,A679498,Teal,2014-05-21 17:10:00,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,5 years,Domestic Shorthair Mix,White,2014-05-22 17:57:00,05/21/2009,Return to Owner,NaN,Neutered Male,5 years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124115,A791609,NaN,2019-03-28 12:47:00,1811 Mernes Meadow in Austin (TX),Stray,Normal,Cat,Intact Male,5 days,Domestic Medium Hair Mix,Blue/White,2019-06-22 17:30:00,03/23/2019,Adoption,Foster,Neutered Male,2 months
124116,A788848,NaN,2019-02-10 00:07:00,10233 Beard Ave in Austin (TX),Stray,Injured,Cat,Unknown,2 years,Domestic Shorthair,Black,2019-02-10 12:19:00,02/10/2017,Transfer,Partner,Unknown,2 years
124117,A767878,Bonnie,2018-03-08 14:17:00,13201 Dessau Road in Austin (TX),Stray,Injured,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Brown Tabby/White,2018-03-09 14:22:00,03/08/2017,Return to Owner,NaN,Spayed Female,1 year
124118,A755860,*Raven,2017-08-10 07:12:00,6106 Begonia Circle in Austin (TX),Stray,Normal,Cat,Intact Female,4 weeks,Domestic Medium Hair Mix,Blue,2017-09-28 18:51:00,07/10/2017,Adoption,NaN,Spayed Female,2 months


## LongBeach

In [27]:
longbeach = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/LongBeach.csv')

In [28]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

rename_map = {
    "Animal Name": "Name",
    "Primary Color": "Color",
    "Sex": "Sex upon Outcome",
    "DOB": "Date of Birth",
    "Intake Date": "Intake_DateTime",
    "Outcome Date": "Outcome_DateTime",
    "Crossing": "Found Location",
}
longbeach = longbeach.rename(columns=rename_map)

for col in ["Date of Birth", "Intake_DateTime", "Outcome_DateTime"]:
    if col in longbeach.columns:
        longbeach[col] = pd.to_datetime(longbeach[col], errors="coerce")

def age_str(from_date, to_date):
    if pd.isna(from_date) or pd.isna(to_date):
        return pd.NA
    if to_date < from_date:
        return pd.NA
    rd = relativedelta(to_date, from_date)
    if rd.years and rd.years > 0:
        return f"{rd.years} year" + ("s" if rd.years != 1 else "")
    months = rd.years * 12 + rd.months
    if months and months > 0:
        return f"{months} month" + ("s" if months != 1 else "")
    weeks = (to_date - from_date).days // 7
    if weeks and weeks > 0:
        return f"{weeks} week" + ("s" if weeks != 1 else "")
    days = (to_date - from_date).days
    return f"{days} day" + ("s" if days != 1 else "")

longbeach["Age upon Intake"] = longbeach.apply(
    lambda r: age_str(r.get("Date of Birth"), r.get("Intake_DateTime")), axis=1
)
longbeach["Age upon Outcome"] = longbeach.apply(
    lambda r: age_str(r.get("Date of Birth"), r.get("Outcome_DateTime")), axis=1
)

if "intake_is_dead" in longbeach.columns:
    longbeach = longbeach[longbeach["intake_is_dead"].astype(str).str.strip().str.lower() != "dead on intake".lower()]

if "outcome_is_dead" in longbeach.columns:
    longbeach = longbeach[~longbeach["outcome_is_dead"].astype(bool)]
if "was_outcome_alive" in longbeach.columns:
    longbeach = longbeach[longbeach["was_outcome_alive"] != 0]

cols_to_drop = [
    "Reason for Intake", "Jurisdiction", "geopoint", "intake_duration",
    "is_current_month", "latitude", "longitude", "Intake Subtype",
    "intake_is_dead", "outcome_is_dead", "was_outcome_alive",
]
existing_to_drop = [c for c in cols_to_drop if c in longbeach.columns]
longbeach = longbeach.drop(columns=existing_to_drop, errors="ignore")

longbeach.to_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/processed/longbeach.csv', index=False)
longbeach

,Animal ID,Name,Animal Type,Color,Secondary Color,Sex upon Outcome,Date of Birth,Intake_DateTime,Intake Condition,Intake Type,Outcome_DateTime,Found Location,Outcome Type,Outcome Subtype,Age upon Intake,Age upon Outcome
0,A705745,*OAKLEY,DOG,GRAY,TRICOLOR,Neutered,2021-08-30,2023-08-30,NORMAL,STRAY,2023-09-03,+2400 BLK E 15TH LONG BEACH CA 90804,ADOPTION,NaN,2 years,2 years
1,A700840,*PIPPY,CAT,BLACK,WHITE,Male,2023-06-01,2023-06-14,UNDER AGE/WEIGHT,STRAY,2023-06-15,+4900 BLK GRISHAM AVE LONG BEACH 90805,RESCUE,LITTLEPAWS,1 week,2 weeks
2,A593145,NaN,CAT,GRAY,NaN,Female,2017-05-08,2017-07-08,FERAL,STRAY,2017-07-24,"0 BLK E MARKET ST, LONG BEACH, CA 90805",RESCUE,CATPAWS,2 months,2 months
4,A675456,*COLORADO,DOG,WHITE,BROWN,Spayed,NaT,2022-05-22,NORMAL,STRAY,2022-06-01,"0 BLK JUNIPERO AVE, LONG BEACH, CA 90803",FOSTER TO ADOPT,NaN,<NA>,<NA>
5,A608487,*TAYLOR,CAT,GRAY,NaN,Spayed,2017-05-27,2018-05-27,NORMAL,OWNER SURRENDER,2018-06-08,"0 BLK W HOME ST, LONG BEACH, CA 90805",ADOPTION,PFE/PAWSHP,1 year,1 year
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51386,A616147,*FERN,DOG,WHITE,NaN,Spayed,2011-10-11,2018-10-11,NORMAL,STRAY,2018-10-19,"XIMENO AVE / E 10TH ST, LONG BEACH, CA 90804",ADOPTION,WALKIN,7 years,7 years
51387,A628288,NaN,CAT,TAN,WHITE,Female,2019-03-14,2019-06-14,UNDER AGE/WEIGHT,STRAY,2019-06-28,"XIMENO AVE / E ANAHEIM ST, LONG BEACH, CA 90804",ADOPTION,WALKIN,3 months,3 months
51388,A599587,NaN,CAT,SEAL PT,NaN,Female,2014-11-02,2017-11-02,NORMAL,STRAY,2017-11-09,"XIMENO AVE / E ATHERTON ST, LONG BEACH, CA 90815",TRANSFER,SPCALA,3 years,3 years
51389,A591021,SANTOS,DOG,BLUE MERLE,WHITE,Male,2012-05-29,2017-05-29,INJURED MODERATE,STRAY,2017-05-31,"XIMENO AVE / E LOS COYOTES DIAGONAL, LONG BEAC...",RETURN TO OWNER,WALKIN,5 years,5 years


## SanJose

In [ ]:
sanjose_2526 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_2526.csv')
sanjose_2425 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_2425.csv')
sanjose_2324 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_2324.csv')
sanjose_2233 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_2233.csv')
sanjose_2122 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_2122.csv')
sanjose_2021 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_2021.csv')
sanjose_1920 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_1920.csv')
sanjose_1819 = pd.read_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/raw/SanJose_1819.csv')
sanjose = pd.concat([
    sanjose_2526, sanjose_2425, sanjose_2324, sanjose_2233, sanjose_2122,
    sanjose_2021, sanjose_1920, sanjose_1819
], ignore_index=True)
sanjose

In [33]:
rename_map = {
    "AnimalName": "Name",
    "PrimaryColor": "Color",
    "AnimalID": "Animal ID",
    "PrimaryBreed": "Breed",
    "Sex": "Sex upon Outcome",
    "DOB": "Date of Birth",
    "IntakeDate": "Intake_DateTime",
    "OutcomeDate": "Outcome_DateTime",
    "Crossing": "Found Location",
    "IntakeCondition": "Intake Condition",
    "IntakeType": "Intake Type",
    "OutcomeType": "Outcome Type",
    "OutcomeSubtype": "Outcome Subtype",
    "SecondaryColor": "Secondary Color",
    "AnimalType": "Animal Type"
}
sanjose = sanjose.rename(columns=rename_map)

for col in ["Date of Birth", "Intake_DateTime", "Outcome_DateTime"]:
    if col in sanjose.columns:
        sanjose[col] = pd.to_datetime(sanjose[col], errors="coerce")

def age_str(from_date, to_date):
    if pd.isna(from_date) or pd.isna(to_date):
        return pd.NA
    if to_date < from_date:
        return pd.NA
    rd = relativedelta(to_date, from_date)
    if rd.years and rd.years > 0:
        return f"{rd.years} year" + ("s" if rd.years != 1 else "")
    months = rd.years * 12 + rd.months
    if months and months > 0:
        return f"{months} month" + ("s" if months != 1 else "")
    weeks = (to_date - from_date).days // 7
    if weeks and weeks > 0:
        return f"{weeks} week" + ("s" if weeks != 1 else "")
    days = (to_date - from_date).days
    return f"{days} day" + ("s" if days != 1 else "")

sanjose["Age upon Intake"] = sanjose.apply(
    lambda r: age_str(r.get("Date of Birth"), r.get("Intake_DateTime")), axis=1
)
sanjose["Age upon Outcome"] = sanjose.apply(
    lambda r: age_str(r.get("Date of Birth"), r.get("Outcome_DateTime")), axis=1
)

# 刪除 OutcomeCondition 為 DEAD 的資料
if "OutcomeCondition" in sanjose.columns:
    sanjose = sanjose[sanjose["OutcomeCondition"].str.strip().str.upper() != "DEAD"]

cols_to_drop = [
    "IntakeReason", "Jurisdiction", "geopoint", "intake_duration",
    "is_current_month", "Intake Subtype","OutcomeCondition", "LastUpdate",
    "Age"
]
existing_to_drop = [c for c in cols_to_drop if c in sanjose.columns]
sanjose = sanjose.drop(columns=existing_to_drop, errors="ignore")

sanjose.to_csv('/Users/zhengherong/Desktop/大學/大五上/資科社/Final Project/fa-25-econ-5166-group-5/data/processed/sanjose.csv', index=False)
sanjose

,Animal ID,Name,Animal Type,Color,Secondary Color,Breed,Sex upon Outcome,Date of Birth,Intake_DateTime,Intake Condition,Intake Type,IntakeSubtype,Outcome_DateTime,Outcome Type,Outcome Subtype,Found Location,Age upon Intake,Age upon Outcome
0,A0075579,BAILEY,DOG,BLACK,RED,LABRADOR RETR,SPAYED,1994-01-16,2024-10-15,MED R,STRAY,OTC,2024-10-15,RTO,NaN,SENTER RD X TULLY RD,30 years,30 years
2,A0533827,PATCHES,DOG,TRICOLOR,BLACK,PARSON RUSS TER,NEUTERED,2006-02-06,2024-08-28,MED SEV,EUTH REQ,NaN,2024-08-28,RTO,NaN,NaN,18 years,18 years
5,A0569573,SHALE,CAT,BLACK,NaN,DOMESTIC SH,SPAYED,2007-10-12,2024-09-25,MED SEV,STRAY,OTC,2024-09-25,EUTH,NaN,1600 BLOCK ALMADEN RD,16 years,16 years
7,A0608333,BLACK,CAT,TORBI-BRN,NaN,DOMESTIC SH,SPAYED,2008-05-05,2024-08-31,MED R,STRAY,OTC,2024-09-03,RTF,NaN,RIVER VIEW DR,16 years,16 years
9,A0636780,SPONGIE,DOG,FAWN,NaN,POODLE MIN,NEUTERED,NaT,2024-07-22,MED SEV,STRAY,OTC,2024-07-22,RTO,NaN,SNELL AVE,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18167,A1387338,TUFFY,DOG,WHITE,NaN,CHIHUAHUA SH,MALE,2023-06-30,2025-06-30,HEALTHY,STRAY,FIELD,2025-07-01,RTO,NaN,THE WOODS DR / SNELL AVE,2 years,2 years
18169,A1387344,NaN,CAT,GRAY,WHITE,DOMESTIC SH,UNKNOWN,2025-05-10,2025-06-30,MED SEV,STRAY,FIELD,2025-07-02,EUTH,NaN,RUBY AVE / ABORN RD,1 month,1 month
18170,A1387466,NaN,CAT,GRAY,WHITE,DOMESTIC SH,MALE,2025-06-02,2025-06-30,MED SEV,STRAY,MEDVET,2025-07-02,DIED,NaN,MIGUELITA RD,4 weeks,1 month
18171,A1392279,NaN,CAT,TABBY-BRN,WHITE,DOMESTIC SH,NEUTERED,2023-06-12,2025-06-09,MED R,STRAY,OTC,2025-06-18,RTF,NaN,CARIBE WAY/RINEHART WAY,1 year,2 years


## 整合

In [ ]:
# Add Location column
austin_df["Location"] = "Austin"
longbeach["Location"]    = "Long Beach"
sanjose["Location"]     = "San Jose"

# Get union of all columns
all_cols = sorted(set(austin_df.columns) | set(longbeach.columns) | set(sanjose.columns))

# Reindex to union
df_austin_u = austin_df.reindex(columns=all_cols)
df_lbc_u    = longbeach.reindex(columns=all_cols)
df_sj_u     = sanjose.reindex(columns=all_cols)

# Concatenate
df_merged = pd.concat([df_austin_u, df_lbc_u, df_sj_u], ignore_index=True)

# Save
df_merged.to_csv(OUT, index=False)

# Schema presence check
schema_matrix = pd.DataFrame({
    "Column": all_cols,
    "Austin": [c in austin_df.columns for c in all_cols],
    "Long Beach": [c in longbeach.columns for c in all_cols],
    "San Jose": [c in sanjose.columns for c in all_cols],
})

# Summaries
summary = {
    "austin_rows": austin_df.shape[0],
    "longbeach_rows": longbeach.shape[0],
    "sanjose_rows": sanjose.shape[0],
    "merged_rows": df_merged.shape[0],
    "merged_cols": df_merged.shape[1],
    "output_path": OUT
}

# Display preview and schema
display_dataframe_to_user("Merged_3cities_preview", df_merged.head(50))
display_dataframe_to_user("Schema_matrix_latest", schema_matrix)